### ***Importando as bibliotecas***

In [1]:
from datasets import load_dataset
from unsloth import FastLanguageModel

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### ***Inicializando o Foundation Model***

O Foundation Model escolhido para o treinamento será o **unsloth/tinyllama-bnb-4bit** por conta do tamanho e a utilização da memória.

In [2]:
max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


### ***Pré-processamento do dataset***

In [3]:
EOS_TOKEN = tokenizer.eos_token

**Prompt**

Para estruturar os dados de treinamento vai ser utilizado o Alpaca, com isso vai ser possível fornecer ao modelo uma estrutura consistente que vai incluir:
- **Instrução**
- **Input (Contexto adicional)**
- **Resposta esperada**

In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

**Funções de apoio**

As funções abaixo serão utilizadas para o pre-processamento do dataset, uma função valida a quantidade de vazios dentro de um campo especifico e a outra vai formatar os dados do dataset para o prompt.

In [5]:
# Verifica a quantidade de vazio em um campo específico do dataset
def contar_campos_vazios(dataset, campo):
    qtd_vazio = 0

    for example in dataset:
        if not example[campo]:
            qtd_vazio += 1

    return qtd_vazio

In [6]:
# Formata o dataset para o formato esperado pelo modelo
def formatting_prompts_func(examples):
    inputs = examples["title"]
    outputs = examples["content"]
    texts = []
    
    for input, output in zip(inputs, outputs):     
        instruction = "Generate a detailed product description based on the following title:"
        
        # Formata o prompt incluindo o token EOS ao final
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}

**Formatando o dataset**

Para a formatação será utilizado as duas funções acima.

Inicialmente será validado quantos campos vazios existem na coluna 'content' e na coluna 'title' do dataset, depois será removido esses vazios e finalmente será realizado a formatação dos dados e a criação de uma nova coluna chamada 'text' que vai conter o prompt que será utilizado no treinamento.

In [7]:
# Carrega o dataset completo.
dataset = load_dataset("json", data_files='./LF-Amazon-1.3M/treino/trn.json', split="train")

# Valida a quantidade de linhas com 'content' vazio
qtd_vazio = contar_campos_vazios(dataset, 'content')
print(f"Linhas com content vazio: {qtd_vazio}")

# Valida a quantidade de linhas com 'title' vazio
qtd_vazio = contar_campos_vazios(dataset, 'title')
print(f"Linhas com title vazio: {qtd_vazio}")

# Filtrar o dataset, removendo linhas com 'title' ou 'content' vazios
dataset_processado = dataset.filter(lambda example: example["content"] != "" and example["title"] != "")

# Agora que o dataset está limpo, vamos formatar os prompts
dataset_formatado = dataset_processado.map(formatting_prompts_func, batched=True)

Linhas com content vazio: 749901
Linhas com title vazio: 126834


### ***Validando quantidade de tokens***

Após o dataset estar formatado vou realizar uma contagem de tokens para saber, na média, quantos tokens estão sendo usados no meu campo 'text'.

Essa validação é apenas para ter uma ideia de qual o valor de **max_seq_length** eu posso utilizar ao carregar e treinar os modelos.

In [8]:
# Função para contar tokens
def contar_tokens(exemplo):
    tokens = tokenizer.encode(exemplo["text"], truncation=False)  # Tokenizar o campo 'text'
    return {"num_tokens": len(tokens)}  # Retorna o número de tokens

In [9]:
# Aplicar a função de contagem de tokens ao dataset formatado
dataset_com_tokens = dataset_formatado.map(contar_tokens)

# Calcular a média de tokens por exemplo
media_tokens = sum(dataset_com_tokens["num_tokens"]) / len(dataset_com_tokens)
print(f"Média de tokens por exemplo: {media_tokens}")

Média de tokens por exemplo: 253.46525647600012


### ***Separando o dataset***

Para finalizar será verificado a quantidade de dados que o dataset possui após o pre-processamento e será realizado um **shuffle** nestes dados para depois separar o dataset em 10 partes iguais e para cada parte será salvo um dataset no formato json.

Foi utilizado o **shuffle** para que cada parte separado do dataset possua informações de produtos diferentes.

In [10]:
# Validando quantidade de dados no dataset
print(f"Quantidade de dados no dataset: {len(dataset_formatado['text'])}")

Quantidade de dados no dataset: 1390403


In [11]:
# Aplicando a randomização no dataset
dataset_randomizado = dataset_formatado.shuffle(seed=42)

# Verificar os primeiros exemplos para garantir que o shuffle foi aplicado
print(dataset_randomizado[:5])

{'uid': ['0385753845', 'B00FXK0XLS', '0967020506', '1285424484', 'B0046TT1NI'], 'title': ['How to Babysit a Grandma', "Earth's Essentials Organic Fractionated Coconut Oil-16 Oz. Pump Bottle-USP Food Grade", 'Cold Cash: The Perfect Heist (Northern Mania!)', 'South-Western Federal Taxation 2014: Corporations, Partnerships, Estates &amp; Trusts', 'Set of 3 Black Minicheck Country Homespun Dish Towels'], 'content': ["PreS-Gr 2&#x2014;In a companion to Reagan's How to Babysit a Grandpa (Knopf, 2012), a young girl heads over to her grandma's house for a sleepover babysitting session-with the child providing clear and humorous instructions to readers on how to care for a grandma. The to-do list contains many choices for Grandma to select from, including a walk to the park, reading, taking photos, playing dress-up, and adding sugary sprinkles to her meal items. The child wisely allows plenty of time for Grandma to look at the pages while reading a book, peek at the stars, and choose the best s

In [12]:
# Dividir o dataset em 10 partes iguais usando 'shard'
dataset_divido = [dataset_randomizado.shard(num_shards=10, index=i) for i in range(10)]

# Salvar as partes
for i, ds in enumerate(dataset_divido):
    print(f"Tamanho da parte {i+1}: {len(ds)}")
    
    ds.to_json(f"./LF-Amazon-1.3M/treino/trn_formatado_parte_{i+1}.json")

Tamanho da parte 1: 139041


Creating json from Arrow format:   0%|          | 0/140 [00:00<?, ?ba/s]

Tamanho da parte 2: 139041


Creating json from Arrow format:   0%|          | 0/140 [00:00<?, ?ba/s]

Tamanho da parte 3: 139041


...

Creating json from Arrow format:   0%|          | 0/140 [00:00<?, ?ba/s]

Tamanho da parte 9: 139040


Creating json from Arrow format:   0%|          | 0/140 [00:00<?, ?ba/s]

Tamanho da parte 10: 139040


Creating json from Arrow format:   0%|          | 0/140 [00:00<?, ?ba/s]